# Learning a Restricted Boltzmann Machine with sampling
In addition to combinatorial optimization problems, QUBO can also be used to compute a Boltzmann machine, which use probability distribution models of problem including global or local optimum.  
Here we review the sampling method in Boltzmann machine with a simple model.

## What is RBM (Restricted Boltzmann Machine)?
RBM (Restricted Boltzmann Machine) is a model with a restricted network structure for a stochastic network model called a Boltzmann machine.

reference:
Restricted Boltzmann machine
https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine

## Reference Materials
It all started with this paper from the University of Montreal.  
A well-known resercher in deep learning presented the following paper as a possible implementation of RBM in D-Wave, and its implementation on D-Wave machines was considered.

On the Challenges of Physical Implementations of RBMs
Vincent Dumoulin and Ian J. Goodfellow and Aaron Courville and Yoshua Bengio
https://arxiv.org/pdf/1312.5258.pdf

A policy was explored that sampling could be used to estimate the gradient of the RBM NLL(negative logarithmic likelihoods) needed for training using a software simulator that mimics D-Wave, but the actual machine was not used.  
This paper and the following article by Dr. Geoffrey Hinton are very helpful in learning about RBM.

A Practical Guide to Training Restricted Boltzmann Machines
Geoffrey Hinton
https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf

## Advocacy of Boltzmann sampling
Next, as a result of actually running the above problems on a D-Wave machine,

Application of Quantum Annealing to Training of Deep Neural Networks
Steven H. Adachi, Maxwell P. Henderson
https://arxiv.org/abs/1510.06356

This takes a specific policy of using the quantum annealer as a Boltzmann sampling machine to help in the estimation of the NLL described above, and the training is actually being done on an actual machine.

## RBM Model Overview
First, the model consists of two layers, called the visible layer and the hidden layer. It is an undirected graph with no directional coupling.

<img src="https://github.com/Blueqat/Blueqat-tutorials/blob/master/tutorial-ja/img/021_0.png?raw=1">

The probability distribution follows a Boltzmann distribution as shown below.

$$p(v,h) = \frac{1}{Z}exp(-E(v,h))$$

This probability distribution is specified by the energy function and is shown below for the number of nodes in the visible layer n and the number of nodes in the hidden layer m.

$$E(v,h) = -\sum_{i=1}^n b_i v_i - \sum_{j=1}^m c_j h_j -\sum_{i=1}^n\sum_{j=1}^m W_{ij}v_ih_j$$

The normalization constants (distribution functions) is as follows.

$$Z = \sum_{v_k}\sum_{h_l}exp\left( \sum_kb_kv_k + \sum_l c_l h_l + \sum_{k,l} W_{kl}v_k h_l \right)$$

From the complete bipartite graph, the conditional probability distributions are as follows using the sigmoid function for $v$ and $h$, respectively.

$$p(h_j = 1|v) = sigm(c_j+\sum_iW_{ij}v_i) \\p(v_i = 1|h) = sigm(b_i+\sum_jW_{ij}h_j)$$

## About Learning
Next, we want to see how to train the RBM model consisting of the above probability distribution.  
Even with DBM (Deep Boltzmann Machine), having multi-layers, the learning is done in the form of RBM.  
These trainings perform error calculations on the training data and the model so as to maximize the log-likelihood $\log{P}$.  
The gradient calculation of the coupling coefficients and bias is shown below using $\log{P}$.

<div>
<img src="https://github.com/Blueqat/Blueqat-tutorials/blob/master/tutorial-ja/img/021_1.png?raw=1" width=800>
</div>

Here, there is not a very efficient calculation of the expected value of the model in the gradient calculation of the coupling coefficient above.

<div>
<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=%3D%20%5Cfrac%7B1%7D%7BZ%7D%5Csum_%7B%5C%7Bv_k%5C%7D%7D%5Csum_%7B%5C%7Bh_l%5C%7D%7Dv_ih_j%20exp%5Cleft(%5Csum_kb_kv_k%20%2B%20%5Csum_lc_lh_l%2B%5Csum_%7Bkl%7DW_%7Bkl%7Dv_kh_l%5Cright)%0D%0A" width=500>
</div>

In practice, it is very difficult to obtain this value directly, so the Gibbs sampling method, such as the CD method, is used to calculate the hidden layer and the visible layer in order to obtain the value.  
The CD method is fairly approximate to save computational cost, and improving the accuracy of this method is computationally expensive and time-consuming.

The above is an update of the coupling factor, but the same is true for the bias update.

## Parameter updates using Boltzmann sampling
Here, instead of this CD method, we're going to use an annealing machine to calculate the gradient, which is the basis of the Boltzmann sampling study using the actual machine.

The coupling coefficients and bias update equations when using Boltzmann sampling are as follows.  
$\alpha$ is momentum and $\epsilon$ is learning rate.

<dev>
    <img src="https://github.com/Blueqat/Blueqat-tutorials/blob/master/tutorial-ja/img/021_3.png?raw=1" width=700>
</dev>

After learning the RBM, back-propagation study with a classical calculator is done to finish it off.

## Sampling methods using annealing

The D-Wave machine is built on the theory of quantum annealing, which is basically designed to find a minimum value to solve an optimization problem.
In practice, however, the effects of the external environment and other reasons often prevent us from settling on an optimal solution.
Therefore, the basis of sampling learning is to use this property of falling into a local solution as a sampling machine to find a distribution.

If we can assume that the variability of the excited states is a Boltzmann distribution, we can approximate it to the following equation.
The correspondence between the following equation and Boltzmann distribution above allow us to introduce the sampling method into the update equation.

<dev>
    <img src="https://github.com/Blueqat/Blueqat-tutorials/blob/master/tutorial-ja/img/021_4.png?raw=1" width="250">
</dev>

$H_f$ is the cost function to be found final, and $\beta_{eff}$ is the variable that adjusts the distribution of the sampling.  
By using it, we can approximate the most computationally intensive part as follows

<dev>
    <img src="https://github.com/Blueqat/Blueqat-tutorials/blob/master/tutorial-ja/img/021_5.png?raw=1" width=500>
</dev>

We apply this to the model expectation values.  
The same applies to model expectations for other visible and hidden layers.

## Run with a simple example.
Let's learn about the above sampling for RBM in a simple example.  
We'll start with a simple question, which is also listed in the D-Wave examples.

<dev>
    <img src="https://github.com/Blueqat/Blueqat-tutorials/blob/master/tutorial-ja/img/021_6.png?raw=1" width=150>
</dev>

In the case of this QUBOmatrix, the cost function is

<dev>
<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=E(x)%20%3D%20-x_1-x_2%2B2x_1x_2" width=200>
<\dev>

If we look for the energy in the number of cases of x, we see that

<dev>
    <img src="https://github.com/Blueqat/Blueqat-tutorials/blob/master/tutorial-ja/img/021_7.png?raw=1" width=150>
</dev>

Thinking analytically, we can get the numbers for all cases here, so we first find the normalized constant Z.

<dev>
    <img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=Z%20%3D%20%5Csum%20exp(E(x))%20%3D%20exp(0)%20%2B%20exp(1)%20%2B%20exp(1)%20%2B%20exp(0)%20%3D%201%2B2.718%2B2.718%2B1%20%3D%207.44" width=700>
</dev>

and

<dev>
    <img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=%7BP%20%3D%20%5Cfrac%7B1%7D%7BZ%7Dexp(E)%0D%0A%7D" width=120>
</dev>

so probability is calculated as 0.13 and 0.37.

<dev>
    <img src="https://github.com/Blueqat/Blueqat-tutorials/blob/master/tutorial-ja/img/021_8.png?raw=1" width=200>
</dev>

It's hard to get this out by hand, so I'll try to think of it as a sampling.

## Sampling with Wildqat
Sampling is easy. Run the above QUBO multiple times and take a distribution.

In [3]:
!pip install -U blueqat

     |████████████████████████████████| 51kB 1.6MB/s 


In [0]:
import blueqat.wq as wq
import numpy as np
a = wq.Opt()

beta = 0.05
a.R = 0.5

a.qubo = np.asarray([[-1,2],[0,-1]])*beta
qarr = [[0,0],[0,1],[1,0],[1,1]]

In [5]:
cnt = [0]*4

for i in range(1000):
    b = a.sa()
  
    for j in range(4):
        if b == qarr[j]:
            cnt[j] += 1

print(cnt)

[104, 385, 406, 105]


The above shows the probability of occurrence of qarr = [[0,0],[0,1],[1,0],[1,1]], with the lower energy being more likely to appear.  

Since the total number of trials was 1,000 this time, we can find the probability distribution by multiplying 1/1000 to each number of appearance.
This is put into the update formula to update the coupling coefficineces.